In [3]:
import twint
import nest_asyncio
import pandas as pd
nest_asyncio.apply()

# pd.set_option("display.max_columns", 999)

# NLTK VADER for sentiment analysis
from nltk.sentiment.vader import SentimentIntensityAnalyzer

import numpy as np

import yfinance as yf

import matplotlib.pyplot as plt
%matplotlib inline
from collections import Counter

import scipy
from sklearn import preprocessing

# granger causality analysis 
import statsmodels.api as sm
from statsmodels.tsa.stattools import grangercausalitytests

# source: https://github.com/twintproject/twint

In [4]:
# Configure
c = twint.Config()

# tesla & TSLA | &zm | $ual | $siri | $pcar | $nvda | $wba | $sbux
c.Search = "tsla"
# c.Since = '2020-12-27'
# c.Until = '2018-12-07'

# c.Debug = True
# c.Limit = 10

# problematic because most people do not show where their geolocation is --> data loss
# c.Near = "NYC"
# c.Geo="40.730610,-73.935242,500km" 

c.Language = "en"


# c.Until = "2020-05-01 00:00:00"


# filter verified: in order to solve the data abundance problem (too much data -> too much time -> a lot of noise in the data) I turned on verification. This leads to a skew towards newspaper and "famous" people. 
# c.Verified = False

## filter popular tweets - 1.Option: min_likes, min_retweets, min_replies - 2.Option: popular-tweets (pt)
# c.Min_likes = 10
# c.Min_retweets = 10
# problem with 10 -> 12 -> 9
# minimum replies eliminates most of the noise from the data, since usually bots or scammers do not comment much - it's easier to like or retweet 
# c.Min_replies = 5

# not working!
# c.Pt = True

# not working!
# c.Filter_retweets = False



# c.Stats = True


c.Store_json = True
# c.Custom_json = ["id", "user_id", "created_at", "date", "tweet", "timezone", "hashtags"]
c.Output = "/Users/pietj.ginski/Desktop/BWL-Studium/BWL 6 Semester/Bachelor Thesis/Raw Data BT/archive-2/twint_msft_minreplies.json"

# Run 
twint.run.Search(c)

1437670029311586325 2021-09-14 08:50:20 +0200 <KarimPabani1> @Talga_Ltd @ABB_Mining Already 96% Lower carbon emissions than all other suppliers. Taking into account Carbon Taxes. @talga  must be the lowest cost, highest quality anode producer in the world. @Tesla @VWGroup @AustinTeslaClub  will all have @talga in their tank.
1437669839930265600 2021-09-14 08:49:35 +0200 <DogeMusician> Tonight!!!! @elonmusk should join and consider to adapt the tangle by @iota for dogecoin and his companies @Tesla and others
1437669643515138049 2021-09-14 08:48:48 +0200 <Mojop4hit> @BabyDogeCoin @Tesla  https://t.co/FLctZXhbKk
1437669508085280769 2021-09-14 08:48:15 +0200 <BLWilli27455086> @stella59260051 @dailypornvidzzz @laciegal2019 @MyFuckableWife @daddysissues18 @BabesLover_____ @WWEDivaBoobs @innocent_wife @Tesla Nice
1437669397355696131 2021-09-14 08:47:49 +0200 <3TriggersTrader> my trading strategy is about being patient and waiting for the price  to go below &amp; then above the buy zone  " my 

1437665659995897859 2021-09-14 08:32:58 +0200 <mat_singleton> @moylecroft @VedaPrime A direct impact of Giga Berlin not being up and operational as originally expected. Europe are getting the Ys likely earmarked for Asia-Pacific Region, which since they’re a larger market @Tesla are prioritizing their sales instead.
1437665611245658119 2021-09-14 08:32:46 +0200 <DenaliCapital> @jurabilis @BradMunchen It’s all orchestrated- Musk / $TSLA is getting hammered by DOJ/SEC OR regulators and he is walking things back
1437665533583843334 2021-09-14 08:32:28 +0200 <vgcrutchfield> @stekkerauto Stash keeps trying to encourage me to buy more bonds and emerging markets.  I am seriously thinking of dumping any stock that has not made money for me and buy $TSLA, just to see what my diversity balance looks like...😂Trading stocks for fun and apparently profit.
1437665353304268800 2021-09-14 08:31:45 +0200 <ActionTrading1> $TSLA Hourly (Sep/13) 👏👏👏👏👏👏👏  https://t.co/i4c9dLb88P
1437665213273280512 2021-09

1437662353101934598 2021-09-14 08:19:50 +0200 <freshjiva> $TSLA ranking #6 in the $SPX by market cap is a reflection of the market's expectation for their EVA.  https://t.co/2jV6sc49z8
1437662318024794112 2021-09-14 08:19:41 +0200 <ApeSharma> $LCID #AdamJonas  @MorganStanley Contd.. * Leadership Ex $TSLA $AAPL Chief Engineer/Chief Designer leadership. * FSD $INTC or $AAPL(rumor)? * Expansion * 11K order many paid lot of advance * County backing * Big retail support * Patents  You are a $TSLA bull hence $LCID bear.
1437662313163546626 2021-09-14 08:19:40 +0200 <Leo88Williams> @Nergock @harrytiffanyiv @Tesla @elonmusk Future revenue lost is just as important as revenue lost today.
1437662098889134085 2021-09-14 08:18:49 +0200 <Leo88Williams> @Nergock @harrytiffanyiv @Tesla @elonmusk Actually, they are. Imagine two people in line. One person leaves the line. The next person takes the product the first person was going to buy, but overall, they sold half as much. They lost a sale. Demand m

1437659138926940163 2021-09-14 08:07:03 +0200 <SjvTesla> @TeslaOwnersEBay @Tesla @elonmusk Lol true
1437659084598259712 2021-09-14 08:06:50 +0200 <grif_j> @SMacweenie @NJLocalRealtor @elonmusk @Tesla @TeslaSolarglass "I have everything but workers...." "they didn't come last week when promised"  Is it that you can't read? Or are you just twisting the narrative to suit?
1437659064381546496 2021-09-14 08:06:46 +0200 <Leo88Williams> @Nergock @Pslavi @Tesla @elonmusk I may end up doing exactly this.
1437659020647665666 2021-09-14 08:06:35 +0200 <TeslaAllTheWay> @TeslaJavi @GreatPaul_Smith @iliketeslas …And he is gone. Always return the favour and block them. $tsla $tslaq  https://t.co/j49yd3WfJE
1437658981053259783 2021-09-14 08:06:26 +0200 <Leo88Williams> @petteri_bergius @Tesla @elonmusk Trade in values are guaranteed for at least 30 days once assigned. I had a trade in value of 53,800 assigned with damage to rear bumper. I then fixed the damage at Tesla certified shop which they directe

1437655589627199488 2021-09-14 07:52:57 +0200 <TheBlakeCheck> @SjvTesla @WholeMarsBlog @Android @Tesla They accept food stamps on the Google play store now so we good.
1437655447603974145 2021-09-14 07:52:23 +0200 <cyclingthealps> @Gfilche @twcat2010 Gali, I started investing in $tsla 4 years ago because of your videos. It was the fundamentals, the numbers, and the technology explained by you that convinced me. I am with #cardano since January and it has the same characteristics.
1437655444479225858 2021-09-14 07:52:22 +0200 <TheDeviantDev> @TESLORD1 @Tesla And it makes driving less boring....
1437655438493896706 2021-09-14 07:52:21 +0200 <elpatriarcasal2> @Numbrs @Tesla @MicroStrategy @Square  https://t.co/RcREUjfdfC
1437655367287193600 2021-09-14 07:52:04 +0200 <a_diyane> @AirdropCamels @Tesla
1437655259204173825 2021-09-14 07:51:38 +0200 <RealKP101> @wendishen99 @DirtyTesla @Tesla I purposely look for places to go to that are at least 1-2 hours away. I dont mind the drive :)
1437655

...

1437248292593692672 2021-09-13 04:54:30 +0200 <Techwor00996445> $BBIG 🚨Worldwide🔥Stock🔥  #Lomotif #USA 225+Million Downloads +@Emmersiveent #NFT  🇨🇳TiktokBanned📵 👀 $mvis $spy $riot $mara $cciv $asts $gme $amc $lotz $tlry $bb $mrin $spce $sprt $ahpi $mrna $nio $li $xpev $bngo $tsla $xl $qs $mrna $nvax $cei #wallstreetbets #Bitcoin #Crypto
1437248285769613312 2021-09-13 04:54:28 +0200 <jsrbbb> @EliBurton_ @StarmanReport @NYKChannel @zubinanary @Tesla Looking forward to Tuesday ❣️🚀
1437248251049164804 2021-09-13 04:54:20 +0200 <Saku20483427> @Tesla  https://t.co/ow97T68n6C
1437248114272899081 2021-09-13 04:53:47 +0200 <IheartTesla> @thriveeachday @Tesla 10 - you must trust FSD. There's no room for second-guessing that can cause errors.
1437248096187076609 2021-09-13 04:53:43 +0200 <Ali_Tesla585> Maybe Tesla won’t open it’s Charging Network unless the EV credits are fair to all parties. @elonmusk #tesla $tsla
1437248049554890756 2021-09-13 04:53:32 +0200 <mooroobee> @iliketeslas Why would 

1437244892212862976 2021-09-13 04:40:59 +0200 <joe_schlindwein> @DimaZeniuk @elonmusk @AustinTeslaClub @WholeMarsBlog @naija_boy09 @Kristennetten @chicago_glenn @Inelonwetrust__ @1stMarsColonist @RationalEtienne @daelmor @Tesla Can we get solar panels installed over those batteries?
1437244798679744512 2021-09-13 04:40:37 +0200 <Stock_Market_Pr> #Gamestop $GME, Cameco, $AAPL, #Tesla $TSLA, AMC: #stocks Buzzing On WallStreetBets Heading Into New Week  https://t.co/nFL4gT3J5V #StockMarket
1437244644459376644 2021-09-13 04:40:00 +0200 <DavidCaoEV> @tesla @elonmusk Is it possible to have computer vision enhanced launch control? For use at a race track where the cameras react to green light and hard launch the car with a quicker reaction time than any human can?
1437244642928390149 2021-09-13 04:40:00 +0200 <Right__Stocks> $penn $mara  $fsr $tsla $bbig $spy $aapl $pltr $sos $srne $snap $fb $coin $btbt $dkng $mvis $sndl $nakd $nio $gme $fubo $amd $amc $aal $coin $wkhs $riot $msft $any $zom  

1437240622486900745 2021-09-13 04:24:01 +0200 <PeterLamariana> Hey @elonmusk @Tesla are an amazing value.   There is a good wait for some models. There are a lot, TOO MANY,  people selling reservations and new cars online.  Can Tesla do something to slow these opportunists down a bit?
1437240585451184129 2021-09-13 04:23:52 +0200 <vamsivanka> @elonmusk Return of Investment on FSD is less than 10 cents to a dollar when you sell the car. Hope I can transfer FSD to another @Tesla  car. Just like iOS Apps to another iPhone at no cost.
1437240099033559043 2021-09-13 04:21:56 +0200 <mediasentiment> $TSLA  new alert at  https://t.co/A7qrDarJHY  #stocks #daytrading  #NYSE #NASDAQ #market 281
1437240085963890691 2021-09-13 04:21:53 +0200 <strive98933134> @Beth_Kindig We are all about the Tesla Mission by buying $TSLA as well as buying Tesla EV’s  &amp; supporting  Elon so I don’t understand your point
1437240082025500673 2021-09-13 04:21:52 +0200 <whinyunderscore> @thriveeachday @Tesla 12
14372

1437237454206128130 2021-09-13 04:11:26 +0200 <Gwen6930> @brockm @elonmusk @Tesla @SpaceX I think it's hilarious you think Elon would help the 90% the wealthy enslaved with the lobbying done to create this mess with financial system to begin with. It's just a repeat of the 1920's crash where keysenian economics helped fixed surface problem but greed is a virus.  https://t.co/TNYf9E7nEv
1437237354339651584 2021-09-13 04:11:02 +0200 <JustinScerini> @AustinTeslaClub @Tesla @itsALLrisky @dogecoin_devs @elonmusk @teslabros @TeslaOwnersEBay @TeslaNY @teslaownersSV @dogeofficialceo @austinbarnard45 @teslaquadcities @gailalfa1 @Kristennetten I like dogs and meme's
1437237271653142529 2021-09-13 04:10:42 +0200 <pandabearchap> 😈👹👺💩 @SECDebHaaland @SECGranholm @EPAMichaelRegan @BrendaMallory46 @GinaMcCarthy46 @AliZaidi46 #Tesla $TSLA @joebiden @potus @teampelosi @senschumer @vp @elonmusk  @Tesla
1437237253399527426 2021-09-13 04:10:38 +0200 <TSLAFanMtl> @RaccoonDBoone @garyblack00 @SawyerMerritt 

KeyboardInterrupt: 

In [ ]:
# *** NLP Function ***

def nlp(data):

# create the analyzer 
    analyzer = SentimentIntensityAnalyzer()

# prepare the data
    sentences = np.array(data["tweet"])

# filter out retweets - we need individual thoughts without any "pre influence"
    data = data[data['retweet']== False]

# get the vader scores 
    data['vader_scores'] = data["tweet"].apply(lambda sentences: analyzer.polarity_scores(sentences))

# append vader_scores 
    data['compound'] = data['vader_scores'].apply(lambda score_dict: score_dict['compound'])
    
# create delta_compound 
    data["delta_compound"] = (data["compound"] / data["compound"].shift(1)) - 1

    return data

In [ ]:
#  *** Engagement Score Function*** 

def engagement_score(df):
# creating a sensible engagement score through normalization
    x = df[['replies_count','retweets_count','likes_count']].values 
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    df_2 = pd.DataFrame(x_scaled)
    engagement_score = df_2[0]+df_2[1]+df_2[2]

# append engagement_score & compound_engagement_score
    df['engagement_score'] = engagement_score
    df['compound_engagement_score'] = df['engagement_score']*df['compound']
    
    return df


In [ ]:
# *** Stock Data Function ***

def yahoo(ticker, start_date, end_date, df_name):

# define the ticker symbol
    tickerSymbol = ticker

# get data on this ticker
    tickerData = yf.Ticker(tickerSymbol)

# get the historical prices for this ticker
    df_name = tickerData.history(period='1d', start=start_date, end=end_date)

# get daily returns
    df_name["daily_returns"] = (df_name["Close"] / df_name["Close"].shift(1)) - 1

# convert index to date_s colums
    df_name['date_s'] = pd.to_datetime(df_name.index)
    
    return df_name

In [ ]:
# *** Group Function ***

def group(data, stock_data):

    data_g = data.groupby(pd.to_datetime(data['date']).dt.date).mean()

# convert to datetime
    data_g['date_s'] = pd.to_datetime(data_g.index)

# # merge the two datasets
    data_g_merge = pd.merge(data_g, stock_data, on='date_s', how='inner')
    
    return data_g_merge
    

In [ ]:
# *** Correlation Analysis Function ***

def correlation(data,column_1,column_2):

    corr, p_value = scipy.stats.pearsonr(data[column_1][1:], data[column_2][1:])

    print('Correlation Coefficient:',corr.round(3))
    print('P-Value:',p_value.round(3))

In [ ]:
# *** Lag Function *** 

def lag(df):

# apple delta return (has to be one day ahead )
    ap_dr = df["Close"].shift(2)[3:]
# apple delta compound 
    ap_dc = df["compound"][3:]


# correlation analysis
    corr, p_value = scipy.stats.pearsonr(ap_dc, ap_dr)

    print('Correlation Coefficient:',corr.round(3))
    print('P-Value:',p_value.round(3))

In [ ]:
# *** Granger Causality Function ***

def granger(data, lag):

    data_2 = data[["Close", "compound"]].pct_change().dropna()
    data_2.replace([np.inf, -np.inf], np.nan, inplace = True)
    data_2 = data_2.dropna()
# # execute granger causality test
    gc_res = grangercausalitytests(data_2, lag)

# Microsoft

### Data Import 

In [12]:
# Import Dataset 

# file name 
file_name = "twint_tesla_verified.json"
string = "/Users/pietj.ginski/Desktop/BWL-Studium/BWL 6 Semester/Bachelor Thesis/Raw Data BT/archive-2/rubbish/{}".format(file_name)
# import the json file 
raw_msft = pd.read_json(string, lines = True)

raw_msft.count()[1]


142102

In [17]:
raw_msft['date'].sort()

AttributeError: 'Series' object has no attribute 'sort'

### Data Manipulation

In [ ]:
# apply nlp()
nlp_msft = nlp(raw_msft)

# apply yahoo()
price_msft = yahoo('MSFT', '2020-06-01','2021-05-31', 'msft')

# apply group()
grouped_msft = group(nlp_msft, price_msft)

# apply engagement_score() 
grouped_msft = engagement_score(grouped_msft)

### Data Analysis

#### Microsoft

In [ ]:
# apply correlation()
correlation(grouped_msft,'Close','compound')

In [ ]:
# apply lag()
lag(grouped_msft)

In [ ]:
# apply granger()
granger(grouped_msft,1)

# Tesla 

In [ ]:
# Import Dataset 

# file name 
file_name = "twint_tsla_verified.json"
string = "/Users/pietj.ginski/Desktop/BWL-Studium/BWL 6 Semester/Bachelor Thesis/Raw Data BT/archive-2/{}".format(file_name)
# import the json file 
raw_msft = pd.read_json(string, lines = True)

raw_msft.count()[1]

In [ ]:
raw_msft.tail()